In [2]:
##import relevant packages and apply settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date as dt
from scipy.stats import chi2_contingency
import requests
import xml.etree.ElementTree as et
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)

The purpose of this file is to explore the initial dataset provided by Global Giving, determine potential variables of interest, complete any data cleaning and feature engineering steps and add new relevant data sets. NOTE: I did a bit of quick replacements in excel prior to importing this data, creating null values for the numerical columns that match the variable's data type (i.e. 01/01/1990, 9999), replacing the 'NULL' string in those columns. This is not especially important here, but became an issue when the data file was uploaded to QuickSight at a later stage. Pandas will automatically recognize NULL values as missing, generally, but this was not the case for QuickSight.

The first few cells are basic data exploration commands to review the file.


In [4]:
#read in clients file (i.e. skills test data set)
file ='clients.csv'
clients = pd.read_csv(file, index_col = 'id')

#set missing values 
clients = clients.replace(9999, np.nan)
clients = clients.replace(1/1/1900, np.nan)
clients = clients.replace('1/1/1900 0:00', np.nan) 

In [5]:
#review first head of the client file
clients.head(10)

,name,status,country_of_registration,year_founded,original_vetting_approval_date,most_recent_vetting_approval_date,vetting_expiration_date,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,earliest_project_created_date,most_recent_project_created_date,earliest_project_deactivation_date,most_recent_project_deactivation_date,last_login,completed_accelerator_program,number_of_field_visits,most_recent_field_visit_date,all_time_funds_raised,all_time_unique_donors,acquisition_source_category,all_time_unque_project_themes,active_contacts
id,,,,,,,,,,,,,,,,,,,,,,,,
8,World Camp,Delinquent - Due Diligence,United States,2001.0,2/26/2013,2/26/2013,2/26/2015,130069.0,0.0,1.0,0.0,5/20/2003 16:16,5/20/2003 16:16,3/17/2005 0:00,3/17/2005 0:00,11/30/2012 13:39,0,NaN,1/1/1900,411.41,5,word_of_mouth,"Climate Action,Education,Ecosystem Restoration...",1
11,Samadhan,Partner,India,1981.0,7/16/2010,10/10/2019,10/10/2021,58078.0,1.0,5.0,12.0,5/21/2003 15:00,12/26/2017 1:38,5/21/2004 0:00,9/28/2012 16:40,9/8/2020 2:22,0,3.0,7/2/2019,49177.39,244,field,"Child Protection,Economic Growth,Education,Gen...",3
12,Ruchika Social Service Organisation,Leader,India,1985.0,11/14/2011,1/6/2020,1/6/2022,407798.0,4.0,4.0,0.0,5/21/2003 15:46,7/3/2019 5:04,9/15/2006 0:00,2/13/2008 0:00,6/17/2020 5:52,0,3.0,6/10/2019,168601.72,423,renewal,"Child Protection,Education,Ecosystem Restorati...",3
15,"Carolina for Kibera, Inc.",Partner,United States,2001.0,1/22/2013,5/13/2020,5/13/2022,889302.0,1.0,1.0,15.0,6/3/2003 16:28,7/20/2020 18:28,1/8/2009 17:51,5/5/2019 0:00,9/2/2020 11:08,0,1.0,4/17/2019,125605.17,573,globalgiving,"Child Protection,Economic Growth,Education,Eco...",1
35,Associacao Saude Crianca,Superstar,Brazil,1991.0,3/12/2008,10/10/2019,10/10/2021,10732.0,2.0,2.0,2.0,7/1/2003 9:56,8/30/2020 22:37,3/17/2005 0:00,7/10/2020 11:33,9/13/2020 11:12,0,2.0,8/12/2015,55856.20,349,field,"Child Protection,Economic Growth,Education,Gen...",2
46,The River Fund,Leader,United States,1990.0,3/11/2008,6/22/2018,11/22/2020,105479.0,3.0,6.0,1.0,7/7/2003 13:00,11/11/2014 15:05,3/17/2005 0:00,6/25/2018 15:11,8/11/2020 10:39,0,5.0,5/23/2018,148852.91,1591,word_of_mouth,"Child Protection,Economic Growth,Education,Gen...",2
56,Abalimi Bezekhaya - Planters of the Home,Leader,South Africa,1982.0,2/15/2007,5/27/2020,5/27/2022,193500.0,1.0,1.0,0.0,7/14/2003 8:44,10/10/2005 12:15,10/14/2011 15:44,10/14/2011 15:44,9/4/2020 6:43,0,3.0,7/25/2017,69430.00,491,renewal,"Climate Action,Education,Ecosystem Restoration...",2
59,Friends of the Disabled,Delinquent - Due Diligence,Nigeria,1990.0,5/31/2007,8/5/2012,8/5/2014,250000.0,0.0,1.0,8.0,7/14/2003 13:18,11/7/2012 6:49,3/17/2005 0:00,2/27/2014 16:52,1/16/2014 10:13,0,1.0,8/1/2011,6416.10,94,misc,"Economic Growth,Education,Gender Equality,End ...",1
60,Nirnaya,Delinquent - Due Diligence,India,1998.0,7/16/2007,2/12/2013,2/12/2015,267430.0,0.0,8.0,3.0,7/14/2003 13:50,10/22/2012 3:42,7/20/2006 0:00,4/25/2013 15:52,1/21/2013 3:52,0,NaN,1/1/1900,36353.88,118,gg_project,"Child Protection,Economic Growth,Ecosystem Res...",1


In [221]:
#review variable names/types/null values
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19754 entries, 8 to 86245
Data columns (total 24 columns):
name                                     19754 non-null object
status                                   19754 non-null object
country_of_registration                  19741 non-null object
year_founded                             19754 non-null int64
original_vetting_approval_date           19754 non-null object
most_recent_vetting_approval_date        19754 non-null object
vetting_expiration_date                  19754 non-null object
approx_annual_budget                     19754 non-null float64
number_of_currently_active_projects      19754 non-null int64
number_of_funded_projects                19754 non-null int64
number_of_retired_projects               19754 non-null int64
earliest_project_created_date            19754 non-null object
most_recent_project_created_date         19754 non-null object
earliest_project_deactivation_date       19754 non-null object
most_recent_

In [6]:
#review basic statistics for numerical variables
clients.describe()

,year_founded,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,completed_accelerator_program,number_of_field_visits,all_time_funds_raised,all_time_unique_donors,active_contacts
count,19284.000000,1.975400e+04,8657.000000,8657.000000,8657.000000,19754.000000,1647.000000,1.975400e+04,19754.000000,19754.000000
mean,1993.267527,2.890816e+07,0.852027,0.752686,0.799815,0.173129,1.511840,1.900337e+04,60.321454,1.443505
std,103.119640,2.002131e+09,3.240574,5.927877,4.940684,0.378369,0.869183,1.339661e+05,327.981227,1.166394
min,0.000000,-1.020609e+06,0.000000,0.000000,0.000000,0.000000,1.000000,-1.758000e+01,0.000000,0.000000
25%,1995.000000,1.438325e+04,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,1.000000
50%,2006.000000,1.000000e+05,0.000000,0.000000,0.000000,0.000000,1.000000,4.001000e+01,1.000000,1.000000
75%,2012.000000,6.310657e+05,1.000000,0.000000,1.000000,0.000000,2.000000,5.800000e+03,17.000000,2.000000
max,2020.000000,2.780000e+11,264.000000,383.000000,254.000000,1.000000,8.000000,1.069578e+07,17346.000000,29.000000


There were a few different data types that needed to be manipulated here, so I separated out the numerical, quantitative and 'date' columns (technically all date columns were strings, but I convert them later. So, here I'm pulling out all of the initial numerical variables in order to view them their means grouped by each of the status types in order take a cursory look at whether any of these variables might be influencing an organization's current status.

In [7]:
#obtain a list of numerical column names
num_cols = clients.describe().columns

#create a dictionary of numerical column names + mean to use to generate aggregate measures
num_dict = dict.fromkeys(num_cols, "mean")

#compare means of numerical variables grouped by "status"
summary = clients.groupby('status').agg(num_dict)
summary['approx_annual_budget'] = summary.approx_annual_budget.astype('int64')
summary['count'] = clients.groupby('status').agg({'status': 'size'})
summary

,year_founded,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,completed_accelerator_program,number_of_field_visits,all_time_funds_raised,all_time_unique_donors,active_contacts,count
status,,,,,,,,,,,
Delinquent - Due Diligence,1987.188917,12732555,0.000291,0.429403,0.971761,0.091697,1.270270,7184.313329,21.889586,1.101055,10044
Emerging,2003.082388,8115702,0.472313,0.006515,0.110749,0.000420,1.000000,4893.032376,3.897921,1.388154,4761
Leader,2000.881178,12239820,1.834737,1.549474,0.584211,0.791798,1.612188,81206.949033,304.011567,2.784437,951
Partner,1992.298349,120153897,1.246486,0.647590,0.421687,0.342558,1.435606,37362.632731,73.952030,1.738007,3252
Superstar,2003.504021,2877606,3.253691,3.045638,2.712752,0.844504,1.819767,108856.023271,567.782842,3.414209,746


Some interesting initial notes--it appears that Delinquent organziations have been established for a longer period of time that any of the other status categories. Oddly enough, their approximate budgets are ranked second in terms of size, which leads me to think that, despite differences, this is probably not a good factor to pay attention to, as Global Giving seems to be serving organizations with both larger and small budgets.

As you might expect, the number of currently active and funded projects is lower than the other groups, with the exception of the Emerging group on funded projects, which is to be expected, given that they haven't obtained full Partner status yet. They also complete the Accelerator Program less often, have less all times funds raised and fewer donors (again, with the exception of the Emerging clients). None of these variables feel like they have 'predictive' power, however, except funds raised and unique donors. As in, if we had real-time data that would allow us to compare the rate of growth for new projects to ensure they appeared to be "on track" for their specific category (perhaps determined by a combination of factors like location and theme), then projects that are not gaining traction could be identified early and we could take steps to rectify it before the funding period ends. 

A couple of other interesting points here--Delinquent organizations have a slightly lower average number of field visits, but it appears that nearly everyone is getting between one and two, so this is probably not a good predictor. Also, Delinquent organizations make up a clear majority of the organizations on the list. I'm actually not sure what to say about this without additional data and background on Global Giving. Specifically, if it appears that there is a pattern in how organizations interact with the platform/program that I might investigate more closely, like whether they onboard and leave quickly, leave after a single poorly funded project, etc. Having information like that from the Program team would enable me to search out and verify patterns of behavior revealed by the data and to build a narrative around this experience. 

Coming up next is more data cleaning and preparation--there were several date columns, all of which loaded into the dateframe as 'object' types. I extracted all of the 'date' columns, modifying the 'last_login' column (the only one that doesn't specifically say 'date' in the title) to pick up all of them. Later I realized that there were two date formats and came back to split the date columns manually into their two different types for manipulations later. This a bit 'quick and dirty' here; might be done in a more reusable way with RegEx.

I also pulled in all the 'object' columns, less the newly identified date columns in order to do review the categorical ('string') variables.

In [8]:
#generate a list of columns w/ datetime information
clients = clients.rename(columns ={'last_login': 'last_login_date'})
dates = clients[clients.columns[clients.columns.to_series().str.contains('date')]]
dates_col = dates.columns
date_col = ['original_vetting_approval_date', 'most_recent_vetting_approval_date',
       'vetting_expiration_date', 'most_recent_field_visit_date']
dt_col = ['earliest_project_created_date',
       'most_recent_project_created_date',
       'earliest_project_deactivation_date',
       'most_recent_project_deactivation_date', 'last_login_date']

#generate a list of columns with categorical information
cats = clients.select_dtypes('object').drop(dates_col, axis = 1)
cats_col = cats.columns


Just a quick view of the categorical variables here, trying to determine where collapsing of variables might be needed in order to make sense of distributions and conduct interpretable chi-square tests later on. 

In [226]:
#review number of unique categories for each categorical variable
for col in cats_col:
    print('VAR: ', col, 'Unique: ', clients[col].nunique())


VAR:  name Unique:  19697
VAR:  status Unique:  5
VAR:  country_of_registration Unique:  175
VAR:  earliest_project_created_date Unique:  8625
VAR:  most_recent_project_created_date Unique:  8627
VAR:  earliest_project_deactivation_date Unique:  3295
VAR:  most_recent_project_deactivation_date Unique:  3070
VAR:  last_login_date Unique:  19017
VAR:  acquisition_source_category Unique:  16
VAR:  all_time_unque_project_themes Unique:  4817


One of the categorical variables that I thought would be more useful collapsed is country_of_registration, so I pulled in a couple of open source datasets from GitHub. Technically, I probably could've found one dataset with all of the information I needed (or just used the second one), 'but' I didn't realize QuickSight would need Latitude and Longitude measures to map anything outside of the U.S., so the second data set was a later addition.

You'll notice a bit of recoding here, as well. Because the Global Giving dataset does not include any consistent measure for country identification (numerical or alpha country codes), I reviewed the set of unmatched countries and hardcoded in country name changes to ensure all countries matched. I also added Kosovo, which was not a part of the open source data file at all.

In [10]:
#collapse countries into regions
#obtain a list of unique countries and corresponding regions/sub-regions
url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
regions = pd.read_csv(url, error_bad_lines = False)

#obtain a list of country codes and corresponding lat/long
url = 'https://gist.githubusercontent.com/tadast/8827699/raw/3cd639fa34eec5067080a61c69e3ae25e3076abb/countries_codes_and_coordinates.csv'
coordinates = pd.read_csv(url, quotechar = '"', sep = ',', skipinitialspace = True, error_bad_lines = False)

#select desired columns from regions and coordinates dataframes
regions = regions[['name', 'region', 'sub-region', 'alpha-3']]
coordinates = coordinates[["Alpha-3 code","Latitude (average)","Longitude (average)"]]


#merge regions and coordinates dateframes
maps = pd.merge(left = regions, right = coordinates, how = 'left', left_on = 'alpha-3', right_on = 'Alpha-3 code')

#update regions dataframe to correspond with GlobalGiving country names
maps = maps.replace(['United States of America', 'Hong Kong', 'United Kingdom of Great Britain and Northern Ireland', 
                        'Virgin Islands (U.S.)', 'Bolivia (Plurinational State of)','Viet Nam', 'Palestine, State of', 
                        'Russian Federation', 'Bahamas', 'Gambia', 'Taiwan, Province of China', 'Timor-Leste', 
                        'Korea, Republic of' , 'Bonaire, Sint Eustatius and Saba', 'Venezuela (Bolivarian Republic of)', 
                        'North Macedonia', 'Czechia', 'Macao', 'Congo', 'Virgin Islands (British)'], 
                        ['United States', 'China-Hong Kong SAR', 'United Kingdom',
                       'Virgin Islands', 'Bolivia', 'Vietnam', 'Palestine', 'Russia',
                       'Bahamas, The', 'Gambia, The', 'China-Taiwan', 'East Timor',
                       'South Korea', 'Caribbean Netherlands', 'Venezuela',
                       'Macedonia, Republic of', 'Czech Republic', 'Macau',
                       'Congo, Republic of the', 'British Virgin Islands'])
kosovo = {'name': 'Kosovo', 'region': 'Europe', 'sub-region': 'Southern Europe'}
maps = maps.append(kosovo, ignore_index = True)

#merge client and regions dataframe
clients = pd.merge(left = clients, right = maps, how = 'left', left_on = 'country_of_registration', right_on = 'name')



This is just that check for unmatched countries the first time--it says none now, of course :) 

In [11]:
#review failed matches
missing = clients.loc[clients['region'].isnull()]
missing.country_of_registration.unique()


array([nan], dtype=object)

In case you'd like to see what the dataset looks like now with the new columns from the two files with geographic information. 

In [229]:
clients.head(10)

,name_x,status,country_of_registration,year_founded,original_vetting_approval_date,most_recent_vetting_approval_date,vetting_expiration_date,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,earliest_project_created_date,most_recent_project_created_date,earliest_project_deactivation_date,most_recent_project_deactivation_date,last_login_date,completed_accelerator_program,number_of_field_visits,most_recent_field_visit_date,all_time_funds_raised,all_time_unique_donors,acquisition_source_category,all_time_unque_project_themes,active_contacts,name_y,region,sub-region,alpha-3,Alpha-3 code,Latitude (average),Longitude (average)
0,World Camp,Delinquent - Due Diligence,United States,2001,2/26/2013,2/26/2013,2/26/2015,130069.0,0,1,0,5/20/2003 16:16,5/20/2003 16:16,3/17/2005 0:00,3/17/2005 0:00,11/30/2012 13:39,0,9999,1/1/1900,411.41,5,word_of_mouth,"Climate Action,Education,Ecosystem Restoration...",1,United States,Americas,Northern America,USA,USA,38.0,-97.0
1,Samadhan,Partner,India,1981,7/16/2010,10/10/2019,10/10/2021,58078.0,1,5,12,5/21/2003 15:00,12/26/2017 1:38,5/21/2004 0:00,9/28/2012 16:40,9/8/2020 2:22,0,3,7/2/2019,49177.39,244,field,"Child Protection,Economic Growth,Education,Gen...",3,India,Asia,Southern Asia,IND,IND,20.0,77.0
2,Ruchika Social Service Organisation,Leader,India,1985,11/14/2011,1/6/2020,1/6/2022,407798.0,4,4,0,5/21/2003 15:46,7/3/2019 5:04,9/15/2006 0:00,2/13/2008 0:00,6/17/2020 5:52,0,3,6/10/2019,168601.72,423,renewal,"Child Protection,Education,Ecosystem Restorati...",3,India,Asia,Southern Asia,IND,IND,20.0,77.0
3,"Carolina for Kibera, Inc.",Partner,United States,2001,1/22/2013,5/13/2020,5/13/2022,889302.0,1,1,15,6/3/2003 16:28,7/20/2020 18:28,1/8/2009 17:51,5/5/2019 0:00,9/2/2020 11:08,0,1,4/17/2019,125605.17,573,globalgiving,"Child Protection,Economic Growth,Education,Eco...",1,United States,Americas,Northern America,USA,USA,38.0,-97.0
4,Associacao Saude Crianca,Superstar,Brazil,1991,3/12/2008,10/10/2019,10/10/2021,10732.0,2,2,2,7/1/2003 9:56,8/30/2020 22:37,3/17/2005 0:00,7/10/2020 11:33,9/13/2020 11:12,0,2,8/12/2015,55856.20,349,field,"Child Protection,Economic Growth,Education,Gen...",2,Brazil,Americas,Latin America and the Caribbean,BRA,BRA,-10.0,-55.0
5,The River Fund,Leader,United States,1990,3/11/2008,6/22/2018,11/22/2020,105479.0,3,6,1,7/7/2003 13:00,11/11/2014 15:05,3/17/2005 0:00,6/25/2018 15:11,8/11/2020 10:39,0,5,5/23/2018,148852.91,1591,word_of_mouth,"Child Protection,Economic Growth,Education,Gen...",2,United States,Americas,Northern America,USA,USA,38.0,-97.0
6,Abalimi Bezekhaya - Planters of the Home,Leader,South Africa,1982,2/15/2007,5/27/2020,5/27/2022,193500.0,1,1,0,7/14/2003 8:44,10/10/2005 12:15,10/14/2011 15:44,10/14/2011 15:44,9/4/2020 6:43,0,3,7/25/2017,69430.00,491,renewal,"Climate Action,Education,Ecosystem Restoration...",2,South Africa,Africa,Sub-Saharan Africa,ZAF,ZAF,-29.0,24.0
7,Friends of the Disabled,Delinquent - Due Diligence,Nigeria,1990,5/31/2007,8/5/2012,8/5/2014,250000.0,0,1,8,7/14/2003 13:18,11/7/2012 6:49,3/17/2005 0:00,2/27/2014 16:52,1/16/2014 10:13,0,1,8/1/2011,6416.10,94,misc,"Economic Growth,Education,Gender Equality,End ...",1,Nigeria,Africa,Sub-Saharan Africa,NGA,NGA,10.0,8.0
8,Nirnaya,Delinquent - Due Diligence,India,1998,7/16/2007,2/12/2013,2/12/2015,267430.0,0,8,3,7/14/2003 13:50,10/22/2012 3:42,7/20/2006 0:00,4/25/2013 15:52,1/21/2013 3:52,0,9999,1/1/1900,36353.88,118,gg_project,"Child Protection,Economic Growth,Ecosystem Res...",1,India,Asia,Southern Asia,IND,IND,20.0,77.0
9,Khulisa Child Nurturing Organization,Delinquent - Due Diligence,South Africa,1997,1/1/1900,1/1/1900,1/1/1900,0.0,0,0,1,7/18/2003 9:15,6/1/2004 12:44,7/23/2010 0:00,7/23/2010 0:00,1/1/1900 0:00,0,9999,1/1/1900,0.00,0,NaN,Education,0,South Africa,Africa,Sub-Saharan Africa,ZAF,ZAF,-29.0,24.0


Honestly, this is just me being curious about how the Global Giving API worked. It might have been a little faster to just manually identify all of the project themes (I wanted to separate out the all_time_unque_themes column; didn't seem particularly useful in the current form). So, literally all I used it for was to generate a list of themes. I did see some other potentially useful information in the API, specifically at the project-level, but I decided I didn't have the time to explore it all for this skills test. One of the things I could do with project-level data, for example, is associated projects of a particular theme with their funding/donor metrics. Because everything is collapsed in the initial dataset that's not possible. 

I've commented everything out here because it doesn't have my API Key, so it'll just throw errors, so I'll just read in the file(s) created by these next steps so we can continue on with this review.

In [20]:
#expand project themes
#import a list of themes from GlobalGiving API
#themes = requests.get("https://api.globalgiving.org/api/public/projectservice/themes?api_key=XXXX")

In [39]:
#generate a list of themes
#response_body_as_xml = et.fromstring(themes.content)
#xml_tree = et.ElementTree(response_body_as_xml)
#root = xml_tree.getroot()

#xmlList = []

#for sitemap in root:
    #children = sitemap.getchildren()
    #xmlList.append(children[1].text)
    

#create columns in client table for each theme

#if all_time_unque_project_themes contains the column name, value is True, otherwise false
#for theme in xmlList:
    #clients[theme] = pd.np.where(clients.all_time_unque_project_themes.str.contains(theme), True, False)

In [14]:
#create a dataframe of popular themes by country
#themes = pd.DataFrame()

#for e in xmlList:
    #themes[e] = clients.groupby('country_of_registration')[e].agg('sum')
    
#themes = themes.transpose()

#export themes file to csv
#themes = themes.to_csv('themes.csv')

In [37]:
#read in clients file (i.e. skills test data set)
themes ='themes.csv'
themes = pd.read_csv(themes)
themes.head(10)

#list of themes
themes_list = list(themes['Unnamed: 0'].unique())

#if all_time_unque_project_themes contains the column name, value is True, otherwise false
for theme in themes_list:
    clients[theme] = pd.np.where(clients.all_time_unque_project_themes.str.contains(theme), True, False)

In [41]:
#generate a new list of categorical columns w/ regions and expanded themes
#generate a list of columns with categorical information
cats = clients.select_dtypes(['object', 'bool']).drop(dates_col, axis = 1).drop(['country_of_registration', 'name_x', 'name_y', 'all_time_unque_project_themes', 'Alpha-3 code', 'alpha-3'], axis = 1)
cats_col = cats.columns
cats_col

Index(['status', 'acquisition_source_category', 'region', 'sub-region',
       'Animal Welfare', 'Child Protection', 'Climate Action',
       'Peace and Reconciliation', 'Disaster Recovery', 'Economic Growth',
       'Education', 'Ecosystem Restoration', 'Gender Equality',
       'Physical Health', 'End Human Trafficking', 'Justice and Human Rights',
       'Sport', 'Digital Literacy', 'Food Security', 'Arts and Culture',
       'LGBTQIA+ Equality', 'COVID-19', 'Clean Water', 'Disability Rights',
       'End Abuse', 'Mental Health', 'Racial Justice', 'Refugee Rights',
       'Reproductive Health', 'Safe Housing', 'Sustainable Agriculture',
       'Wildlife Conservation'],
      dtype='object')

Quick look at the data with the new columns that indicate whether an organization has ever had a project listed under a specific theme.

In [40]:
clients.head(10)

,name_x,status,country_of_registration,year_founded,original_vetting_approval_date,most_recent_vetting_approval_date,vetting_expiration_date,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,earliest_project_created_date,most_recent_project_created_date,earliest_project_deactivation_date,most_recent_project_deactivation_date,last_login_date,completed_accelerator_program,number_of_field_visits,most_recent_field_visit_date,all_time_funds_raised,all_time_unique_donors,acquisition_source_category,all_time_unque_project_themes,active_contacts,name_y,region,sub-region,alpha-3,Alpha-3 code,Latitude (average),Longitude (average),Animal Welfare,Child Protection,Climate Action,Peace and Reconciliation,Disaster Recovery,Economic Growth,Education,Ecosystem Restoration,Gender Equality,Physical Health,End Human Trafficking,Justice and Human Rights,Sport,Digital Literacy,Food Security,Arts and Culture,LGBTQIA+ Equality,COVID-19,Clean Water,Disability Rights,End Abuse,Mental Health,Racial Justice,Refugee Rights,Reproductive Health,Safe Housing,Sustainable Agriculture,Wildlife Conservation
0,World Camp,Delinquent - Due Diligence,United States,2001.0,2/26/2013,2/26/2013,2/26/2015,130069.0,0.0,1.0,0.0,5/20/2003 16:16,5/20/2003 16:16,3/17/2005 0:00,3/17/2005 0:00,11/30/2012 13:39,0,NaN,1/1/1900,411.41,5,word_of_mouth,"Climate Action,Education,Ecosystem Restoration...",1,United States,Americas,Northern America,USA,USA,38.0,-97.0,False,False,True,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Samadhan,Partner,India,1981.0,7/16/2010,10/10/2019,10/10/2021,58078.0,1.0,5.0,12.0,5/21/2003 15:00,12/26/2017 1:38,5/21/2004 0:00,9/28/2012 16:40,9/8/2020 2:22,0,3.0,7/2/2019,49177.39,244,field,"Child Protection,Economic Growth,Education,Gen...",3,India,Asia,Southern Asia,IND,IND,20.0,77.0,False,True,False,False,False,True,True,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Ruchika Social Service Organisation,Leader,India,1985.0,11/14/2011,1/6/2020,1/6/2022,407798.0,4.0,4.0,0.0,5/21/2003 15:46,7/3/2019 5:04,9/15/2006 0:00,2/13/2008 0:00,6/17/2020 5:52,0,3.0,6/10/2019,168601.72,423,renewal,"Child Protection,Education,Ecosystem Restorati...",3,India,Asia,Southern Asia,IND,IND,20.0,77.0,False,True,False,False,False,False,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,"Carolina for Kibera, Inc.",Partner,United States,2001.0,1/22/2013,5/13/2020,5/13/2022,889302.0,1.0,1.0,15.0,6/3/2003 16:28,7/20/2020 18:28,1/8/2009 17:51,5/5/2019 0:00,9/2/2020 11:08,0,1.0,4/17/2019,125605.17,573,globalgiving,"Child Protection,Economic Growth,Education,Eco...",1,United States,Americas,Northern America,USA,USA,38.0,-97.0,False,True,False,False,False,True,True,True,True,True,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,Associacao Saude Crianca,Superstar,Brazil,1991.0,3/12/2008,10/10/2019,10/10/2021,10732.0,2.0,2.0,2.0,7/1/2003 9:56,8/30/2020 22:37,3/17/2005 0:00,7/10/2020 11:33,9/13/2020 11:12,0,2.0,8/12/2015,55856.20,349,field,"Child Protection,Economic Growth,Education,Gen...",2,Brazil,Americas,Latin America and the Caribbean,BRA,BRA,-10.0,-55.0,False,True,False,False,False,True,True,False,True,True,True,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False
5,The River Fund,Leader,United States,1990.0,3/11/2008,6/22/2018,11/22/2020,105479.0,3.0,6.0,1.0,7/7/2003 13:00,11/11/2014 15:05,3/17/2005 0:00,6/25/2018 15:11,8/11/2020 10:39,0,5.0,5/23/2018,148852.91,1591,word_of_mouth,"Child Protection,Economic Growth,Education,Gen...",2,United States,Americas,Northern America,USA,USA,38.0,-97.0,False,True,False,False,False,True,True,False,True,True,True,True,False,False,Tru

In [42]:
#review frequencies for categorical data
for col in cats_col:
    print('VAR:', col, '\n', clients[col].value_counts(dropna = False), '\n')

VAR: status 
 Delinquent - Due Diligence    10145
Emerging                       4793
Partner                        3309
Leader                          976
Superstar                       762
Name: status, dtype: int64 

VAR: acquisition_source_category 
 direct_search        5089
corporate            4410
strategic_partner    1567
blank                1270
misc                 1102
web                  1088
field                 772
word_of_mouth         733
NaN                   721
gg_project            685
google_ads            681
social_media          514
email                 424
renewal               346
globalgiving          309
content_marketing     220
media                  54
Name: acquisition_source_category, dtype: int64 

VAR: region 
 Americas    9435
Africa      4701
Asia        3108
Europe      2545
Oceania      183
NaN           13
Name: region, dtype: int64 

VAR: sub-region 
 Northern America                   7964
Sub-Saharan Africa                 4671
Souther

In reviewing the distribution of categorical variables, we see the same status distribution as earlier and that the majority of organizations come to Global Giving through direct searches and corporate (partnerships?). I'm not too sure what all these categories mean and, because we were told that it's a 'best effort,' I'm essentially ignoring it until I have more information.

The majority of organizations are from the Americas, then Africa. First thing this distribution tells me is that this truly is a 'global' nonprofit, which is great, but also makes learning what could be causing retention issues more difficult. Seeing this distribution makes me even more keen to speak with representatives from each region on the Program team to see if there appear the be different trends in different global areas and suggests segmentation by region in the dashboards will be vital. I also see a few missing values here, which I check in the next step to make sure there weren't any data matching issues. There wasn't--it just looks like a few organizations had no country of registration listed.

I won't go too deep into the themes here, but the distributions are listed above, though they're a bit difficult to compare in that format.

In [45]:
#verify countries with missing regions
print(clients.loc[clients['region'].isnull()].country_of_registration.value_counts(dropna = False))
print(clients.loc[clients['region'].isnull()].name_x.value_counts(dropna = False))

NaN    13
Name: country_of_registration, dtype: int64
UniSol                                                                  1
Boys and Girls Club of Oshkosh                                          1
Global Business Council on HIV/AIDS, Tuberculosis, and Malaria          1
InSTEDD                                                                 1
BRAC UK                                                                 1
The Bahamas Hurricane Restoration  Fund                                 1
Active Minds, Inc.                                                      1
MARA COMMUNITY WELFARE FOR CHILDREN AND WOMEN INNITIATIAVE -TANZANIA    1
Rose Community Foundation                                               1
American Foreign Service Association                                    1
Pamlico Co Disaster Recovery Coalition                                  1
Central European University                                             1
Heart 9/11                                                

I made a regions column that has the sub-regions listed for everywhere except Oceania. I didn't end up using it in the dashboards this time, but it could be useful to have later on.

In [46]:
#create a regions variable with all sub-regions except locations in Oceania
clients.loc[clients.region == 'Oceania', 'regions']  = 'Oceania'
len(clients.loc[clients['regions'] == 'Oceania']) == len(clients.loc[clients['region'] == 'Oceania'])

True

It suddenly occurred to me that I should check for duplicate organizations, so I did that here. You can see that there definitely are some duplicates here, even when taking country of registration into account. I chose not to do anything about that here, which means any calculations for these organizations will average across their two or more observations. 

However, should we move forward with this project, this is something I'd like to figure out. 

In [51]:
#verify that clients do not appear in the dataset multiple times
duplicates = pd.concat(g for _, g in clients.groupby(['name_x', 'country_of_registration']) if len(g) > 1)
print(len(duplicates), len(duplicates.name_x.unique()))

550 275


,name_x,status,country_of_registration,year_founded,original_vetting_approval_date,most_recent_vetting_approval_date,vetting_expiration_date,approx_annual_budget,number_of_currently_active_projects,number_of_funded_projects,number_of_retired_projects,earliest_project_created_date,most_recent_project_created_date,earliest_project_deactivation_date,most_recent_project_deactivation_date,last_login_date,completed_accelerator_program,number_of_field_visits,most_recent_field_visit_date,all_time_funds_raised,all_time_unique_donors,acquisition_source_category,all_time_unque_project_themes,active_contacts,name_y,region,sub-region,alpha-3,Alpha-3 code,Latitude (average),Longitude (average),Animal Welfare,Child Protection,Climate Action,Peace and Reconciliation,Disaster Recovery,Economic Growth,Education,Ecosystem Restoration,Gender Equality,Physical Health,End Human Trafficking,Justice and Human Rights,Sport,Digital Literacy,Food Security,Arts and Culture,LGBTQIA+ Equality,COVID-19,Clean Water,Disability Rights,End Abuse,Mental Health,Racial Justice,Refugee Rights,Reproductive Health,Safe Housing,Sustainable Agriculture,Wildlife Conservation,regions
12794,A-PAD KOREA(Asia Pacific Alliance for Disaster...,Superstar,South Korea,2016.0,9/29/2019,9/29/2019,9/29/2021,269613.0,4.0,0.0,0.0,8/21/2019 2:45,8/12/2020 0:45,10/5/2019 0:00,10/5/2019 0:00,9/11/2020 0:12,1,NaN,1/1/1900,46738.53,236,direct_search,"Child Protection,Disaster Recovery,Economic Gr...",3,South Korea,Asia,Eastern Asia,KOR,KOR,37.0,127.5,False,True,False,False,True,True,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
12795,A-PAD KOREA(Asia Pacific Alliance for Disaster...,Superstar,South Korea,2016.0,9/29/2019,9/29/2019,9/29/2021,269613.0,4.0,0.0,0.0,8/21/2019 2:45,8/12/2020 0:45,10/5/2019 0:00,10/5/2019 0:00,9/11/2020 0:12,1,NaN,1/1/1900,46738.53,236,direct_search,"Child Protection,Disaster Recovery,Economic Gr...",3,South Korea,Asia,Eastern Asia,KOR,KOR,37.0,127.5,False,True,False,False,True,True,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
4921,ADN,Delinquent - Due Diligence,Côte d'Ivoire,1997.0,2/20/2014,2/20/2014,2/20/2016,40000.0,0.0,0.0,3.0,2/20/2014 12:36,1/19/2015 8:24,4/26/2015 17:54,9/17/2015 16:36,2/22/2018 13:15,1,NaN,1/1/1900,5516.18,55,direct_search,"Peace and Reconciliation,Economic Growth,Educa...",0,Côte d'Ivoire,Africa,Sub-Saharan Africa,CIV,CIV,8.0,-5.0,False,False,False,True,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
4922,ADN,Delinquent - Due Diligence,Côte d'Ivoire,1997.0,2/20/2014,2/20/2014,2/20/2016,40000.0,0.0,0.0,3.0,2/20/2014 12:36,1/19/2015 8:24,4/26/2015 17:54,9/17/2015 16:36,2/22/2018 13:15,1,NaN,1/1/1900,5516.18,55,direct_search,"Peace and Reconciliation,Economic Growth,Educa...",0,Côte d'Ivoire,Africa,Sub-Saharan Africa,CIV,CIV,8.0,-5.0,False,False,False,True,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
400,AIP Foundation,Partner,Vietnam,1999.0,6/1/2008,5/17/2018,9/17/2020,2243506.0,3.0,1.0,0.0,3/7/2010 23:41,5/13/2020 3:56,6/7/2012 9:34,6/7/2012 9:34,9/15/2020 21:49,1,3.0,5/17/2016,3721322.37,288,field,"Child Protection,Education,Physical Health",11,Vietnam,Asia,South-eastern Asia,VNM,VNM,16.0,106.0,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
401,AIP Foundation,Partner,Vietnam,1999.0,6/1/2008,5/17/2018,9/17/2020,2243506.0,3.0,1.0,0.0,3/7/2010 23:41,5/13/2020 3:56,6/7/2012 9:34,6/7/2012 9:34,9/15/2020 21:49,1,3.0,5/17/2016,3721322.37,288,field,"Child Protection,Education,Physical Health",11,Vietnam,Asia,South-eastern Asia,VNM,VNM,16.0,106.0,False,True,False,

Here I've moved to converting the date columns to their appropriate format and also calculating a few potentially relevant metrics: year, month and time delta from today for each column.

In [52]:
#convert all date columns to datetime and calculate month and year of each of the datetime activities
#calculate the time passed between each action and today

for col in date_col:
    clients[col] = pd.to_datetime(clients[col], format = '%m/%d/%Y', errors = 'coerce')
    clients[col + '_year'] = clients[col].dt.year
    clients[col + '_month'] = clients[col].dt.month
    clients[col + '_delta'] =pd.to_datetime(dt.today()) - clients[col]
  

for col in dt_col:
    clients[col] = pd.to_datetime(clients[col], format = '%m/%d/%Y %H:%M', errors = 'coerce')
    clients[col + '_year'] = clients[col].dt.year
    clients[col + '_month'] = clients[col].dt.month
    clients[col + '_delta'] =pd.to_datetime(dt.today()) - clients[col]
  


I was also curious about the distance between the vetting_expiration_date and some date variables so I could review things like mean number of days between last login and vetting expiration for Delinquent organizations.

In [53]:
#calculate the number of days between next (or last) vetting approval date and key metrics (lapse_col)
lapse_col = ['last_login_date', 'original_vetting_approval_date', 'most_recent_field_visit_date', 'earliest_project_created_date']

for col in lapse_col:
    clients[col + '_lapse'] = clients['vetting_expiration_date'] - clients[col]

That's all the data preparation I decided to do for now, so I just exported the file and moved to a new notebook to look at some of the correlations because this one is getting a little long :| 

You can take a look at the corr.ipynb file next to follow along with me.

In [245]:
#review columns and drop unneeded columns
clients.columns

#export file
prepared = clients.to_csv('prepared.csv')